# 1 x 2 Probability for Given English Premier League Match

This is a reusable notebook that will calculate probability of a home win, draw, or away win (1 x 2) for any match in the English Premier League. All data will be scraped from Understat

In [2]:
import sys
sys.path.append('../scraper')

from epl_table_scraper import WebScraper

scraper = WebScraper(url="https://understat.com/league/EPL")
scraper.load_page()
scraper.click_element("home-away3")
data_frame = scraper.scrape_table()

scraper.close()

print(data_frame)

scraper is locating element... check
element located:  <selenium.webdriver.remote.webelement.WebElement (session="3a6b1060dc1b3ed101948144506f388a", element="f.51669E5179C52255919E986B32950EA1.d.7A38E97FA116E32AE487263E7BC8D3A1.e.26")>
element html:  <input id="home-away3" type="radio" name="home-away" value="a">
clicked the button
checking table update...
confirmed table updated successfully
found table
closing driver...
driver closed
     №                     Team  M  W  D  L   G  GA PTS          xG  \
0    1                Liverpool  5  4  1  0  10   3  13  11.58+1.58   
1    2        Nottingham Forest  5  3  2  0   8   4  11   9.03+1.03   
2    3                  Chelsea  5  3  1  1  12   5  10   7.91-4.09   
3    4          Manchester City  6  3  1  2  10   7  10  11.96+1.96   
4    5              Aston Villa  6  3  1  2  10  11  10  11.57+1.57   
5    6                   Fulham  6  2  2  2   7   6   8   9.55+2.55   
6    7         Newcastle United  6  2  2  2   8   8   8   9.88+